In [3]:
import torch
import torch.nn as nn
import torchvision
import numpy as np
import  os
import pickle
import dataloadr
import vgg16
import shutil

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

Initializing Datasets and Dataloaders...
Files already downloaded and verified


In [4]:
model = vgg16.vgg16(in_channels=3,num_classes=10)
model.load_state_dict(torch.load('./model/vgg16.model',map_location='cpu'))

Generate model VGG16 


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [5]:

class VggFeatureExtractor(nn.Module):
    def __init__(self, trained_model):
        super(VggFeatureExtractor,self).__init__()
        self.target_layer = nn.Sequential(*list(trained_model.children())[:-1])


    def forward(self, x):
        x = self.target_layer(x)
        return  x


def feature_extractor_v1(imag_root_dir,feature_save_path,name_save,save_container):
    with torch.no_grad():
        img_names = []
        dloader = dataloadr.get_dataloader(root_dir=imag_root_dir,batch_size=40)
        print('loade...')
        count = 0
        for batchimg,img_name in dloader:
                batchimg = batchimg.to('cpu')
                model.forward(batchimg)
                features = model.extract_features
                features = features.detach().numpy()
                features = features.astype(np.float16)
               
                save_container[count:count+features.shape[0],:] = features
                img_names.extend(img_name)
                count += features.shape[0]
                
                print(count)
                
        pickle.dump(img_names,open(name_save,'wb'))
        np.save(feature_save_path,save_container)

In [6]:
rest_save_container = np.zeros((665,512),dtype=np.float16)            
feature_extractor_v1(imag_root_dir=os.path.abspath(os.path.join(os.getcwd(), "../VisualGenome/RestVisualGeneome")),
                     feature_save_path='../result/rest_feature.npy',
                     name_save = '../result/rest_names.pkl',
                     save_container=rest_save_container)

loade...
40
80
120
160
200
240
280
320
360
400
440
480
520
560
600
640
665


In [10]:

std_save_container = np.zeros((273,512),dtype=np.float16)            
feature_extractor_v1(imag_root_dir=os.path.abspath(os.path.join(os.getcwd(), "../VisualGenome/std1000")),
                     feature_save_path='../result/std_feature.npy',
                     name_save = '../result/std_names.pkl',
                     save_container=std_save_container)

loade...
40
80
120
160
200
240
273


In [12]:
def feature_extractor_v2(imag_root_dir,feature_save_path,save_container):
    with torch.no_grad():
        extractor = VggFeatureExtractor(model)
        dloader = dataloadr.get_dataloader(root_dir=imag_root_dir,batch_size=10)
        print('loade...')
        for batchimg,img_name in dloader:
            batchimg = batchimg.to('cpu')
            features = extractor(batchimg)
            features = features.detach().numpy()
            print(img_name)
            break

# Compare

In [13]:
from numpy import linalg

In [14]:
std1000 = np.load('../result/std_feature.npy')
rest = np.load('../result/rest_feature.npy')
std_names = pickle.load(open('../result/std_names.pkl','rb'))
rest_names = pickle.load(open('../result/rest_names.pkl','rb'))

In [15]:
std_names = np.asarray(std_names)
rest_names = np.asarray(rest_names)

In [16]:
def calcu_simi():
    res_dict = {}
    for std_idx, std_item in enumerate(std1000):
        simi_vec = []
        for rest_idx,rest_item in enumerate(rest):
            simi = 1.0/(1.0+linalg.norm(std_item-rest_item))
            simi_vec.append(simi)
        top50 = np.argsort(np.asarray(simi_vec))[::-1][:50]
        top50_images = rest_names[top50]
        res_dict[std_names[std_idx]] = top50_images
    return res_dict

In [17]:
res_dict = calcu_simi()

In [18]:
def save_img(result_dict,root_dir):
    for std_img, simi_imgs in result_dict.items():
        imageName = std_img.split("\\")[-1]
        std_img_name = imageName.split(".")[0]
        #std_img_name = std_img.split('/')[-1].strip('.jpg')
        #print(std_img_name)
        #print(root_dir)
        save_path = os.path.join(root_dir,std_img_name)
        #print(root_dir)
        #print(save_path)
        if not os.path.exists(save_path):
            os.mkdir(save_path)
        shutil.copy(std_img,save_path)
        for item in simi_imgs:
            shutil.copy(item,save_path)
        break



In [19]:
res_dict

{'C:\\Users\\js\\Desktop\\VggSimi\\VisualGenome\\std1000\\1.jpg': array(['C:\\Users\\js\\Desktop\\VggSimi\\VisualGenome\\RestVisualGeneome\\867.jpg',
        'C:\\Users\\js\\Desktop\\VggSimi\\VisualGenome\\RestVisualGeneome\\893.jpg',
        'C:\\Users\\js\\Desktop\\VggSimi\\VisualGenome\\RestVisualGeneome\\432.jpg',
        'C:\\Users\\js\\Desktop\\VggSimi\\VisualGenome\\RestVisualGeneome\\690.jpg',
        'C:\\Users\\js\\Desktop\\VggSimi\\VisualGenome\\RestVisualGeneome\\341.jpg',
        'C:\\Users\\js\\Desktop\\VggSimi\\VisualGenome\\RestVisualGeneome\\986.jpg',
        'C:\\Users\\js\\Desktop\\VggSimi\\VisualGenome\\RestVisualGeneome\\490.jpg',
        'C:\\Users\\js\\Desktop\\VggSimi\\VisualGenome\\RestVisualGeneome\\731.jpg',
        'C:\\Users\\js\\Desktop\\VggSimi\\VisualGenome\\RestVisualGeneome\\401.jpg',
        'C:\\Users\\js\\Desktop\\VggSimi\\VisualGenome\\RestVisualGeneome\\905.jpg',
        'C:\\Users\\js\\Desktop\\VggSimi\\VisualGenome\\RestVisualGeneome\\846.jpg',


In [20]:
save_img(res_dict,root_dir='./results')#如果当前目录没有results的文件夹，请先创建一个